In [9]:
import torch

x = torch.FloatTensor(4)
W = torch.FloatTensor(4, 3)
b = torch.FloatTensor(3)
W

tensor([[0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.],
        [0., 0., 0.]])

In [5]:
def linear_function(x, W, b):
    y = torch.matmul(x, W) + b
    return y

In [6]:
print(f'W:{W.shape}, x:{x.shape}, b: {b.shape}')

W:torch.Size([4, 3]), x:torch.Size([4]), b: torch.Size([3])


In [8]:
y = linear_function(x, W, b)
y

tensor([8.5403e-39, 1.0194e-38, 2.2421e-44])

In [10]:
y.shape

torch.Size([3])

In [15]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=3, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=3, out_channels=10, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(10 * 12 * 12, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        print(f'연산전: {x.size()}')
        x = F.relu(self.conv1(x))
        print(f"conv1 연산 후: {x.size()}")
        x = F.relu(self.conv2(x))
        print(f'conv2 연산후 : {x.size()}')
        x = x.view(-1, 10 * 12 * 12)
        print(f"차원 감소 후: {x.size()}")
        x = F.relu(self.fc1(x))
        print(f'fc1 연산 후: {x.size()}')
        x = F.relu(self.fc2(x))
        print(f'fc2 연산 후: {x.size()}')
        return x

cnn = CNN()
output = cnn(torch.randn(10, 1, 20, 20))

연산전: torch.Size([10, 1, 20, 20])
conv1 연산 후: torch.Size([10, 3, 16, 16])
conv2 연산후 : torch.Size([10, 10, 12, 12])
차원 감소 후: torch.Size([10, 1440])
fc1 연산 후: torch.Size([10, 50])
fc2 연산 후: torch.Size([10, 10])


In [16]:
class CNN2(nn.Module):
    def __init__(self):
        super(CNN2, self).__init__()
        self.max_pool1 = nn.MaxPool2d(kernel_size=2)
        self.max_pool2 = nn.MaxPool2d(kernel_size=2)
        self.fc1 = nn.Linear(10 * 5 * 5, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        print(f'연산전: {x.size()}')
        x = F.relu(self.max_pool1(x))
        print(f'max_pool1 연산후: {x.size()}')
        x = F.relu(self.max_pool2(x))
        print(f'max_pool2 연산후: {x.size()}')
        x = x.view(-1, 10 * 5 * 5)
        print(f'차원 감소후: {x.size()}')
        x = F.relu(self.fc1(x))
        print(f'fc1 연산후: {x.size()}')
        x = self.fc2(x)
        print(f'fc2 연산후: {x.size()}')
        return x

cnn2 = CNN2()
output = cnn2(torch.randn(10, 1, 20, 20))

연산전: torch.Size([10, 1, 20, 20])
max_pool1 연산후: torch.Size([10, 1, 10, 10])
max_pool2 연산후: torch.Size([10, 1, 5, 5])
차원 감소후: torch.Size([1, 250])
fc1 연산후: torch.Size([1, 50])
fc2 연산후: torch.Size([1, 10])


In [18]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

train_data = datasets.MNIST('./data/', train=True, download=True, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081))]
))
train_loader = torch.utils.data.DataLoader(dataset=train_data, batch_size=50, shuffle=True)

test_data = datasets.MNIST('./data/', train=False, transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))]
))
test_loader = torch.utils.data.DataLoader(dataset=test_data, batch_size=50, shuffle=True)

In [19]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=20, kernel_size=5, stride=1)
        self.conv2 = nn.Conv2d(in_channels=20, out_channels=50, kernel_size=5, stride=1)
        self.fc1 = nn.Linear(4 * 4 * 50, 500)
        self.fc2 = nn.Linear(500, 10)
        
    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = F.relu(self.conv2(x))
        x = F.max_pool2d(x, kernel_size=2, stride=2)
        x = x.view(-1, 4 * 4 * 50)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x

In [20]:
cnn = CNN()
criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.SGD(cnn.parameters(), lr=0.01)

In [21]:
cnn.train()
for epoch in range(10):
    for index, (data, target) in enumerate(train_loader):
        optimizer.zero_grad()
        output = cnn(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        
        if index % 100 == 0:
            print("loss of {} epoch, {} index: {}".format(epoch, index, loss.item()))

loss of 0 epoch, 0 index: 2.311591148376465
loss of 0 epoch, 100 index: 1.4453136920928955
loss of 0 epoch, 200 index: 0.6443017721176147
loss of 0 epoch, 300 index: 0.27937498688697815
loss of 0 epoch, 400 index: 0.19198156893253326
loss of 0 epoch, 500 index: 0.38074791431427
loss of 0 epoch, 600 index: 0.2774350047111511
loss of 0 epoch, 700 index: 0.27242350578308105
loss of 0 epoch, 800 index: 0.31759920716285706
loss of 0 epoch, 900 index: 0.13244831562042236
loss of 0 epoch, 1000 index: 0.10725497454404831
loss of 0 epoch, 1100 index: 0.09097827225923538
loss of 1 epoch, 0 index: 0.08145273476839066
loss of 1 epoch, 100 index: 0.332340806722641
loss of 1 epoch, 200 index: 0.042103949934244156
loss of 1 epoch, 300 index: 0.1940106749534607
loss of 1 epoch, 400 index: 0.039032794535160065
loss of 1 epoch, 500 index: 0.10476246476173401
loss of 1 epoch, 600 index: 0.2988612651824951
loss of 1 epoch, 700 index: 0.0373876690864563
loss of 1 epoch, 800 index: 0.15957942605018616
loss 